# LAB 3: Regresja wieloraka

Prognozowanie cen domów (wiele zmiennych)

W tym notatniku wykorzystamy dane dotyczące sprzedaży domów. Przewidywanie ich ceny zostanie dokonane przy pomocy regresji wielorakiej. Pierwsze zadanie dotyczy eksploracji regresji wielorakiej poprzez opracowanie nowych cech i pomiar błędu. W drugim zadaniu zaimplementować należy algorytm spadku gradientu.

     Użyj wbudowanych funkcji tworzenia wykresów (lub w inny sposób), aby obliczyć wagi regresji (współczynniki)
     Biorąc pod uwagę wagi regresji, predyktory i wynik napisz funkcję obliczającą resztkową sumę kwadratów
     Spójrz na współczynniki i zinterpretuj ich znaczenie
     Oceń wiele modeli za pomocą RSS

# Potrzebne biblioteki

In [2]:
import pandas as pd
import numpy as np
# dodac wspolczynik liniowy 

# Wczytanie danych ceny domów 

Zestaw danych pochodzi ze sprzedaży domów w King County, regionie, w którym znajduje się miasto Seattle w stanie Waszyngton.

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
domy = pd.read_csv('kc_house_data.csv',sep=',', dtype=dtype_dict)

In [3]:
domy.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Podziel dane na uczące i testowe.
Używamy seed = 0, aby każdy, kto korzysta z tego notebooka, uzyskał te same wyniki. W praktyce możesz ustawić losowy podział.

In [4]:
y = domy['price']
X = domy.drop(['price'], axis=1)
features = X.columns.values
features 

array(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Nauka modelu regresji wielorakiej

Wykorzystując cechy 'sqft_living', 'bedrooms', 'bathrooms' uczymy nasz model.

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
pinw = np.linalg.pinv(X_train[example_features])
w = np.dot(pinw,y_train)
w.shape

(3,)

Po dopasowaniu modelu możemy wyodrębnić współczynniki (wagi) modelu regresji:

In [7]:
print('Otrzymane współczynniki:', w)

Otrzymane współczynniki: [   308.54966331 -41754.38204544  17354.12106635]


# Przewidywanie wartości modelu

Mając wyliczone parametry modelu napisz funkcję do przewidywania wartości dla zadanego modelu

In [8]:
def predict_output(w,X):
    # return  np.dot(w, X.T)
    return  np.dot(X, w) 


X = X_train[example_features]
prediction = predict_output(w,X)
print(prediction)
print(y)


[376513.94632314 467340.55721729 237087.16846861 ... 641960.83166918
 598953.41599022 606377.44288989]
0        221900.0
1        538000.0
2        180000.0
3        604000.0
4        510000.0
           ...   
21608    360000.0
21609    400000.0
21610    402101.0
21611    400000.0
21612    325000.0
Name: price, Length: 21613, dtype: float64


# Wyliczamy błąd (SSE -  sum of squared estimate of errors)

Teraz, gdy możemy wykonać przewidywania na podstawie modelu, napiszmy funkcję obliczającą RSS modelu. Wykonaj poniższą funkcję, aby obliczy sumę kwadratów błędu estymacji (SSE) na podstawie modelu, danych i wyniku.

In [9]:
def policz_SSE(model, data, outcome):
    # Wylicz przewidywaną wartość
    prediction = predict_output(model,data)
    # Policz błąd bezwzglęny
    ralitive_error = abs(prediction - outcome)
    # Podnieś błędy do kwadratu i zsumuj je
    SSE = sum (ralitive_error**2) 
    return(SSE)    

Przetestuj swoją funkcję obliczając błąd SSE z danych TEST dla przykładowego modelu:

In [10]:
przykladowe_rss = policz_SSE(w, X_test[example_features], y_test)
print(przykladowe_rss) 

261738485259246.44


# Utwórz nowe cechy

Mimo iż nasz model regresji wielorakiej obejmuje wiele różnych cech (np. ilosc_sypiani, powierzchnia i ilosc_lazienek) możemy również rozważyć przekształcenie istniejących cech, np. log(powierzchnia), czy nawet mnożenie ilości sypialni i łazienek.

Użyjemy funkcji logarytmu, aby utworzyć nowe cechy, więc najpierw powinieneś importujemy ją z biblioteki matematycznej.

In [11]:
from math import log


Następnie utwórz następujące 4 nowe cechy jako kolumny w danych TRENINGOWYCH i TESTOWYCH:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + lat 
Jako przykład oto pierwsza:

In [12]:
X_train['bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x**2)
X_test['bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)

In [13]:
# utwórz pozostałe 3 funkcje zarówno w danych TRENINGOWYCH, jak i TESTOWYCH
X_train['bed_bath_rooms'] = X_train['bedrooms'] * X_train['bathrooms']
X_test['bed_bath_rooms'] = X_test['bedrooms'] * X_test['bathrooms']
X_train['log_sqft_living'] = np.log(X_train['sqft_living']) 
X_test['log_sqft_living'] = np.log(X_test['sqft_living'])
X_train['lat_plus_long'] = X_train['lat'] + X_train['lat']
X_test['lat_plus_long'] = X_test['lat'] + X_test['long']

* Kwadrat sypialni zwiększa separację między nielicznymi sypialniami (np. 1) i wieloma sypialniami (np. 4), ponieważ 1 ^ 2 = 1, ale 4 ^ 2 = 16. W konsekwencji ta funkcja będzie miała wpływ głównie na domy z wieloma sypialniami.
* Sypialnia razy łazienka daje tak zwaną funkcję „interakcji”. Wynik jest wysoki, gdy  wartościu *obu* cech są duże.
* Przejęcie logarytmu stóp kwadratowych powoduje zbliżenie dużych wartości i rozłożenie małych wartości. Wynika to z reguły prawoskośności posiadanych danych/posiadanego atrybutu.
* Dodawanie szerokości do długości geograficznej jest całkowicie bezsensowne, ale i tak to zrobimy (zobaczymy później dlaczego)

**Pytanie quizu: Jaka jest średnia (średnia arytmetyczna) twoich 4 nowych funkcji w danych TEST? (w zaokrągleniu do 2 cyfr)**

In [14]:
X_test.iloc[:,-4:].mean().map('{:,.2f}'.format)

bedrooms_squared     12.21
bed_bath_rooms        7.45
log_sqft_living       7.55
lat_plus_long       -74.65
dtype: object

# Uczenie wielu modeli

Teraz poznamy wagi trzech (zagnieżdżonych) modeli do przewidywania cen domów. Pierwszy model będzie miał najmniej cech, drugi model doda jedną cechę, a trzeci doda jeszcze kilka:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: + bedrooms\*bathrooms
* Model 3: + log squarefeet, bedrooms squared, i (bezsensowne) latitude + longitude

In [15]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Teraz, gdy mamy już cechy, poznaj wagi trzech różnych modeli do przewidywania docelowej = „ceny” za pomocą funkcji model_train i spójrz na wartość wag/współczynników:

In [16]:
# Funkcja szukająca współczynników modelu
def model_train(X_train, Y_train):
    # Wylicz współczynniki modelu wykorzystując pseudoodwrotność
    pinw = np.linalg.pinv(X_train)
    model = np.dot(pinw,Y_train)
    return(model)

In [17]:
# Wyucz 3 modele:
model_1 = model_train(X_train[model_1_features],y_train)
model_2 = model_train(X_train[model_2_features],y_train)
model_3 = model_train(X_train[model_3_features],y_train)

In [18]:
# Wyświetl wagi współczynników modelu:
print(model_1)
print(model_2)
print(model_3)


[ 3.00159310e+02 -5.23689819e+04  4.58247068e+03  5.29238529e+05
  2.05262847e+05]
[ 2.92464746e+02 -1.21761358e+05 -1.09383881e+05  5.30036632e+05
  2.03619080e+05  3.35897011e+04]
[ 5.55032435e+02  1.34221113e+03  8.84250144e+04  1.08688195e+05
  1.77276220e+05 -1.60202196e+04  8.18714827e+02 -6.45602857e+05
  2.17376390e+05]


**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 1?**

**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 2?**

Zastanów się, co to znaczy.

Dla pierwszego modelu 1 : dodatni

Dla pierwszego modelu 2 : ujemny

Może to oznaczać, ze wazniejsze dla wzrostu ceny jest interakcja lazienek i pokój, niż sama ilośc lazienek. Współczynik/wagi łazienki jest ujemny w drugim modelu, dlatego, że ilość łazienek jest brana również pod uwagę w współczyniku/wagi bed_bath_rooms.

# Porównywanie wielu modeli

Teraz, gdy otrzymaliśmy trzy modele i wyodrębniliśmy wagi modeli, chcemy ocenić, który model jest najlepszy.

Używając wcześniej naopisanych funkcji obliczy SSE dla danych uczących dla każdego z trzech modeli.

In [19]:
# Policz SSE na danych TRENINGOWYCH dla każdeg z 3 modeli i zapisz wartości:
rss_1_train = policz_SSE(model_1, X_train[model_1_features], y_train)
rss_2_train = policz_SSE(model_2, X_train[model_2_features], y_train)
rss_3_train = policz_SSE(model_3, X_train[model_3_features], y_train)
print(rss_1_train)
print(rss_2_train)
print(rss_3_train)


1082381909711006.1
1065791753438922.6
991825573399597.5


**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TRENINGOWYCH?** Czy tego się spodziewałeś/-łaś?

Najniszy poziom ma model 2, spodziewał sie modelu 3. Wynika to z faktu, że w modelu 3 jest za dużo  atrybutów zależnych od siebie. 

Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli.

In [20]:
# Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli i zapisz wartości:
rss_1_test = policz_SSE(model_1, X_test[model_1_features], y_test)
rss_2_test = policz_SSE(model_2, X_test[model_2_features], y_test)
rss_3_test = policz_SSE(model_3, X_test[model_3_features], y_test)
print(rss_1_test)
print(rss_2_test)
print(rss_3_test)

233700269953885.84
229575356886062.9
5.885254952052211e+18


**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TESTOWYCH?** Czy tego się spodziewałeś/-łaś? Pomyśl o cechach, które zostały dodane do każdego z modeli.

Również model drugi, wynika z faktu, że interakcja miedzy ilościa lazienek a pokojami jest bardzo ważna.

# Policz pochodną

Przejdziemy teraz do obliczania pochodnej funkcji kosztu regresji. Przypomnij sobie, że funkcja kosztu jest sumą kwadratów różnic między punktami danych a przewidywanym wynikiem.

Ponieważ pochodna sumy jest sumą pochodnych, możemy obliczyć pochodną dla pojedynczego punktu danych, a następnie zsumować na podstawie punktów danych. Możemy zapisać kwadratową różnicę między obserwowanym a przewidywanym wynikiem dla pojedynczego punktu w następujący sposób:

(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)^2

Gdzie mamy *k* cech i stałą. Tak więc pochodną w odniesieniu do wagi w[i] według reguły łańcucha jest:

2\*(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)\* [cecha_i]

Pojęcie w nawiasach to tylko błąd (różnica między prognozowaniem a wyjściem). Możemy więc ponownie napisać to jako:

2\*error\*[cecha_i]

Oznacza to, że pochodną wagi cechy *i* jest suma (ponad punkty danych) 2-krotności iloczynu błędu i samej cechy. W przypadku stałej jest to tylko dwukrotność sumy błędów!

Przypomnijmy, że dwukrotność sumy iloczynu dwóch wektorów jest tylko dwukrotnością iloczynu dwóch wektorów. Dlatego pochodna wagi dla *cechy_i* jest tylko dwukrotnością iloczynu między wartościami *cechy_i* a bieżącymi błędami.

Mając to na uwadze, należy wykonać następującą funkcję pochodną, która oblicza pochodną współczynnika na podstawie wartości cechy (we wszystkich punktach danych) i błędów (we wszystkich punktach danych).

In [21]:
def feature_derivative(errors, feature):
    # Oblicz podwojony iloczyn cech i błędów, a następnie zwróć otrzymną wartość
    derivative = sum(2* errors * feature)
    return(derivative)

# Metoda spadku gradientu / gradientu prostego (Gradient Descent)

Teraz napiszemy funkcję, która wykonuje spadek gradientu. Podstawową przesłanką jest prosta. Biorąc pod uwagę punkt początkowy, aktualizujemy bieżące wagi, przesuwając się w kierunku ujemnego gradientu. Przypomnijmy, że gradient jest kierunkiem *wzrostu*, a zatem gradient ujemny jest kierunkiem *spadku* i staramy się *zminimalizować* funkcję kosztu.

Współczynnik, z jakim poruszamy się w kierunku gradientu ujemnego, nazywa się „rozmiarem kroku” - $\alpha$. Zatrzymujemy się, gdy jesteśmy „wystarczająco blisko” do rozwiązania optymalnego. Definiujemy to, wymagając, aby wielkość (długość) wektora gradientu była mniejsza niż stała „tolerancja” - $\epsilon$.

Mając to na uwadze, wykonaj poniższą funkcję spadku gradientu poniżej, używając powyższej funkcji pochodnej. Dla każdego kroku zejścia gradientu aktualizujemy wagę każdej funkcji przed obliczeniem naszych kryteriów zatrzymania

In [22]:
from math import sqrt # wywołanie funkcji na wektorze [g[0], g[1], g[2]] daje sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [23]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # miej pewność, że wagi są typu numpy array
    while not converged:
        # obliczyć prognozy na podstawie feature_matrix i wag za pomocą funkcji predict_output()
        predict_output_value =  predict_output(weights,feature_matrix)
        # wylicz błąd predykcji jako: predict_output - output
        errors = predict_output_value - output
    
        gradient_sum_squares = 0 # inicjuj sumę pierwiastków gradientów
        # chociaż nie osiągnęliśmy jeszcze tolerancji, zaktualizuj wagę każdej funkcji
        for i in range(len(weights)): # zapętlenie każdej wagi
            # Przypomnij sobie, że feature_matrix [:, i] to kolumna cech powiązana z wagami[i]
            # obliczyć pochodną dla wagi[i]:
            derivative = feature_derivative(errors, feature_matrix.iloc[:,i])
            # dodaj kwadrat wartości pochodnej do sumy kwadratów gradientów (ocena zbieżności) 
            gradient_sum_squares += derivative**2  
            # odejmij wielkość kroku pomnożoną przez pochodną od bieżącej wagi
            weights[i] -= step_size * derivative
        # oblicz pierwiastek kwadratowy z sumy gradientów kwadratów, aby uzyskać wielkość gradientu:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

Kilka rzeczy, na które należy zwrócić uwagę, zanim zaczniemy korzystać z metody spadku gradientu. Ponieważ gradient jest sumą wszystkich punktów danych i obejmuje iloczyn błędu i wartości cechy, sam gradient będzie bardzo duży, ponieważ cechy są duże (stopy kwadratowe), a wynik jest duży (ceny). Tak więc, chociaż można oczekiwać, że „tolerancja” będzie niewielka, mała jest jedynie zależna od wielkości cechy.

Z podobnych powodów rozmiar kroku będzie znacznie mniejszy niż można się spodziewać, ale dzieje się tak, ponieważ gradient ma tak duże wartości.

# Uruchamianie spadku gradientu jako prostej regresji

Chociaż opadanie gradientu jest zaprojektowane dla regresji wielorakiej, ponieważ stała jest teraz funkcją, możemy użyć funkcji spadku gradientu do oszacowania parametrów prostej regresji na "squarefeet". Następująca komórka ustawia funkcję parametr_macierz, wynik, wagi początkowe i rozmiar kroku dla pierwszego modelu:

In [24]:
# test spadku gradientów
simple_features = ['sqft_living']
my_output = 'price'
# (simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
simple_feature_matrix = X_train[simple_features]
simple_feature_matrix.insert(0,'const',1.0,)   # Dodanie 
output = y_train
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Następnie uruchom gradient z powyższymi parametrami.

In [25]:
simple_weights = [float(f'{x:.01f}') for x in regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)]
simple_weights

[-46999.9, 283.5]

Porównaj wagi do tych uzyskanych przy pomocy pseudoodwrotności?

In [26]:
simple_weights_reverse = [float(f'{x:.01f}') for x in model_train(simple_feature_matrix,output)]
print('pseudo-reverse weights: ',simple_weights_reverse)
print('gradent desent weights: ',simple_weights)
x_test_simple_features = X_test[simple_features]
x_test_simple_features.insert(0,'const',1)
print('pseudo-reverse SSE:',policz_SSE(simple_weights_reverse,x_test_simple_features,y_test) )
print('gradent desent SSE:',policz_SSE(simple_weights,x_test_simple_features,y_test))

pseudo-reverse weights:  [-48257.1, 284.0]
gradent desent weights:  [-46999.9, 283.5]
pseudo-reverse SSE: 267780323695881.94
gradent desent SSE: 267741860253500.94




**Pytanie quizu: Jaka jest waga sqft_living - drugi element „simple_weights” (w zaokrągleniu do 1 miejsca po przecinku)?**

In [27]:
simple_weights[1]

283.5

Zobacz jak się zachowuje metoda gradientowa po przeskalowaniu wartości cech:

In [28]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

simple_features = ['sqft_living']
my_output = 'price'
domy_sc  = pd.DataFrame(data= scaler.fit_transform(domy[['sqft_living','price']]),columns= ['sqft_living','price']) # min max skalowanie  
X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(domy_sc[simple_features],  domy_sc[my_output], test_size=0.2, random_state=0)
X_train_sc.insert(0,'const',1)
initial_weights = np.random.rand(2)
step_size = 0.00001
tolerance = 0.1

simple_weights = [float(f'{x:.03f}') for x in regression_gradient_descent(X_train_sc, y_train_sc, initial_weights, step_size, tolerance)]
simple_weights_reverse = [float(f'{x:.03f}') for x in model_train(X_train_sc,y_train_sc)]
print('pseudo-reverse weights: data scaled',simple_weights_reverse)
print('gradent desent weights: data scaled ',simple_weights)

X_test_sc.insert(0,'const',1)
print('pseudo-reverse SSE: data scaled ', policz_SSE(simple_weights_reverse,X_test_sc,y_test_sc) )
print('gradent desent SSE: data scaled ',policz_SSE(simple_weights,X_test_sc,y_test_sc))
simple_weights


pseudo-reverse weights: data scaled [-0.005, 0.493]
gradent desent weights: data scaled  [-0.005, 0.494]
pseudo-reverse SSE: data scaled  4.60933738467692
gradent desent SSE: data scaled  4.612724979461017


[-0.005, 0.494]

Skalowanie danych daje nastepujace zalety: 
 - Łatwiejszy dobór parametrów poczatkowych  (initial_weights,step_size, tolerance), szczegółnie initial_weights, bo można wziać losowe liczby zakresie 0-1 
 - Szybsze, obliczanie grandietu 